<h1> Polynomiality of the ML-degree of generic linear concentration models </h1>
Author: Tim Seynnaeve <br>
Version: 1.0 <br>
This Sage worksheet accompanies the paper <br>
"COMPLETE QUADRICS: SCHUBERT CALCULUS FOR GAUSSIAN MODELS AND SEMIDEFINITE PROGRAMMING" <br>
by LAURENT MANIVEL, MATEUSZ MICHALEK, LEONID MONIN, TIM SEYNNAEVE, AND MARTIN VODIČKA <br>
Available on <a href="https://www.orlandomarigliano.com/lssm"> https://www.orlandomarigliano.com/lssm </a>

References: <br>
LLT: D. Laksov, A. Lascoux, and A. Thorup. On Giambelli’s theorem on complete correlations. Acta Math., 
162(3-4):143–199, 1989. <br>
NRS: J. Nie, K. Ranestad, and B. Sturmfels. The algebraic degree of semidefinite programming. Math. Program., 122(2, Ser. A):379–405, 2010. <br>
MMW: M. Michalek, L. Monin, and J. Wiśniewski. Maximum likelihood degree and space of
orbits of a C∗ action. arXiv preprint arXiv:2004.07735, 2020. <br>
SU: B. Sturmfels and C. Uhler. Multivariate Gaussian, semidefinite matrix completion, and convex
algebraic geometry. Ann. Inst. Statist. Math., 62(4):603–638, 2010. <br>

<h3>Sections 2,3,4: Polynomiality of ML-degree</h3>

In [9]:
#psi(I) computes the Lascoux coefficient (Def 2.4, or [LLT, A.15]),
#where I is a strictly increasing sequence of nonnegative integers
#The computation is done recursively, based on [LLT, A.15.3 and A.15.5].
coefDict = {():1}
def psi(I):
    n=len(I)
    if tuple(I) in coefDict.keys():
        return coefDict.get(tuple(I))
    else:
        if n%2==0:
            if n==2:
                antw=sum([binomial(I[0]+I[1],i) for i in [I[0]+1..I[1]]])
            else:
                antw=sum([(-1)^(k+1)*psi([I[0],I[k]])*psi(I[1:k]+I[k+1:]) for k in [1..n-1]])
        else:
            if n==1:
                antw=2^(I[0])
            else:
                antw=sum([(-1)^(k)*psi([I[k]])*psi(I[:k]+I[k+1:]) for k in [0..n-1]])
        coefDict[tuple(I)]=antw
        return antw

In [10]:
#Example 2.5:
#(If the last coefficient doesn't match: this is a typo in the paper, to be corrected in next version.)
print([psi([0,2]),psi([0,3]),psi([1,2]),psi([1,3]),psi([2,3])])

[3, 7, 3, 10, 10]


In [11]:
#psiPoly(I) is the polynomial from Theorem 4.3: psi of complement of $I \subset [n]$, viewed as a polynomial in n
#It is computed recursively, as in the first proof of Theorem 4.3
#Some auxiliary functions...
R.<n> = PolynomialRing(QQ)
def summationPolynomial(f,n0):
    d=f.degree()
    return R.lagrange_polynomial([(i,sum([f(j) for j in [n0..i]])) for i in [n0..n0+d+1]])
def isIncreasing(L):
    for i in [0..len(L)-2]:
        if L[i]>=L[i+1]:
            return false
    return true
from itertools import product
s1 = set((0, 1))
polyDict = {():1+R.0-R.0,tuple([0]):R.0} #There really should be a better way to convince sage that 1 is a polynomial in R, not an integer
def psiPoly(I):
    r=len(I)
    if tuple(I) in polyDict.keys():
        return polyDict.get(tuple(I))
    else:
        if I[0]==0:
            antw = (n-r+1)*psiPoly(I[1:])-2*sum([psiPoly(I[1:j]+[I[j]+1]+I[j+1:]) for j in [1..r-2] if I[j+1]>I[j]+1])-2*psiPoly(I[1:r-1]+[I[r-1]+1])
        else:
            S=set(product(s1, repeat = r))
            S.remove(tuple([0 for i in range(r)]))
            sumList = [[I[i]-el[i] for i in range(r)] for el in S]
            sumList = [el for el in sumList if isIncreasing(el)]
            g=sum([psiPoly(el) for el in sumList])
            antw=summationPolynomial(g,0)(n-1)
        polyDict[tuple(I)]=antw
        return antw

In [12]:
#deltaPoly(m,s) is \delta(m,n,n-s), viewed as a polynomial in n
#computed using theorem 3.8
def deltaPoly(m,s):
    L=[list(el) for el in Partitions(m-s, length=s, max_slope=-1)]+[list(el)+[0] for el in Partitions(m-s, length=s-1, max_slope=-1)]
    for el in L:
        el.reverse()
    return sum([psi(I)*psiPoly(I) for I in L])

In [13]:
#phiPoly(d) is \phi(n,d), viewed as a polynomial in n
#computed using (3.3), see also Theorem 2.2
def phiPoly(d):
    antw =0
    s=0
    while binomial(s+1,2) <= d:
        antw=antw+s*deltaPoly(d,s)
        s=s+1
    return antw/n

In [6]:
#We can compute the polynomials phi(d) very quickly. 
#Compare with [SU: section 2.2] and [MMW: Theorem 2.7 and Conjecture 5.1]
for d in [1..50]:
    print("phi(n,"+str(d)+")="+str(factor(phiPoly(d))))

phi(n,1)=1
phi(n,2)=n - 1
phi(n,3)=(n - 1)^2
phi(n,4)=(5/6) * (n - 2) * (n - 1) * (n - 3/5)
phi(n,5)=(7/12) * (n - 2) * (n - 1) * (n^2 - 19/7*n + 6/7)
phi(n,6)=(43/120) * (n - 2) * (n - 1) * (n^3 - 221/43*n^2 + 316/43*n - 60/43)
phi(n,7)=(1/5) * (n - 3) * (n - 2) * (n - 1) * (n^3 - 19/4*n^2 + 27/4*n - 5/6)
phi(n,8)=(29/280) * (n - 3) * (n - 2) * (n - 1) * (n^4 - 218/29*n^3 + 585/29*n^2 - 1844/87*n + 140/87)
phi(n,9)=(169/3360) * (n - 3) * (n - 2) * (n - 1) * (n^5 - 1770/169*n^4 + 551/13*n^3 - 14042/169*n^2 + 12136/169*n - 560/169)
phi(n,10)=(8357/362880) * (n - 3) * (n - 2) * (n - 1) * (n^6 - 114126/8357*n^5 + 629471/8357*n^4 - 1816902/8357*n^3 + 2911016/8357*n^2 - 2201088/8357*n + 60480/8357)
phi(n,11)=(9053/907200) * (n - 4) * (n - 3) * (n - 2) * (n - 1) * (n^6 - 118395/9053*n^5 + 625700/9053*n^4 - 1749975/9053*n^3 + 2847707/9053*n^2 - 2352810/9053*n + 37800/9053)
phi(n,12)=(40993/9979200) * (n - 4) * (n - 3) * (n - 2) * (n - 1) * (n^7 - 685483/40993*n^6 + 4763290/40993*n^5 - 1799575

phi(n,28)=(161622044185347541/5444434725209176080384000000) * (n - 6) * (n - 5) * (n - 4) * (n - 3) * (n - 2) * (n - 1) * (n^21 - 13537490781453873342/161622044185347541*n^20 + 536327339450715052151/161622044185347541*n^19 - 13371369101736083422794/161622044185347541*n^18 + 235455918475417219649442/161622044185347541*n^17 - 3116006236125081609967836/161622044185347541*n^16 + 32206915612050300256229182/161622044185347541*n^15 - 266896215511381548894368148/161622044185347541*n^14 + 1807936548660186327554701673/161622044185347541*n^13 - 10165019365169563182405402870/161622044185347541*n^12 + 48020298726375566176750719243/161622044185347541*n^11 - 192241318016911683660856825122/161622044185347541*n^10 + 654244522512868452437275902064/161622044185347541*n^9 - 1886334264723583247117894427984/161622044185347541*n^8 + 4572325099786289879477471345264/161622044185347541*n^7 - 9286619685461737645895817896736/161622044185347541*n^6 + 16057391376963298780729705553280/161622044185347541*n^5 - 245497

phi(n,34)=(52186344919471715951861/8683317618811886495518194401280000000) * (n - 7) * (n - 6) * (n - 5) * (n - 4) * (n - 3) * (n - 2) * (n - 1) * (n^26 - 5945873214077431546955860/52186344919471715951861*n^25 + 323894937743906651776558190/52186344919471715951861*n^24 - 11235453717911214552150669832/52186344919471715951861*n^23 + 278906950102585427944472308619/52186344919471715951861*n^22 - 5279048584113993454825569480124/52186344919471715951861*n^21 + 79265137538137184296000344861800/52186344919471715951861*n^20 - 969780051508985569307578572493792/52186344919471715951861*n^19 + 9853462492299709525180019600986379/52186344919471715951861*n^18 - 84312779423481430813362986746128844/52186344919471715951861*n^17 + 614047763293113761495184609129330950/52186344919471715951861*n^16 - 3839270737932365295677370177426854632/52186344919471715951861*n^15 + 20766414234053344147682011322128437989/52186344919471715951861*n^14 - 97904121979936355441276135321504145124/52186344919471715951861*n^13 + 40529

phi(n,38)=(103934245966183708231997989/6881876545613172523157989790790451200000000) * (n - 8) * (n - 7) * (n - 6) * (n - 5) * (n - 4) * (n - 3) * (n - 2) * (n - 1) * (n^29 - 13812676201242831069852750022/103934245966183708231997989*n^28 + 881831398044718086572315025369/103934245966183708231997989*n^27 - 36024270630939037315780679376270/103934245966183708231997989*n^26 + 1058410089828351032537848112169885/103934245966183708231997989*n^25 - 23835047429563143719240143247039190/103934245966183708231997989*n^24 + 428223042662548848898910780970528765/103934245966183708231997989*n^23 - 6308642404044842618993674279399100070/103934245966183708231997989*n^22 + 77741470679032042589664179465058848835/103934245966183708231997989*n^21 - 813389516942662222483876524992577095370/103934245966183708231997989*n^20 + 7308179423444646879031131902489836741275/103934245966183708231997989*n^19 - 56879673431520624625695351956258452016970/103934245966183708231997989*n^18 + 386046438412557093047794989771040691042

phi(n,42)=(10169562831441321164789189207/334525266131638071081700620534407516651520000000) * (n - 8) * (n - 7) * (n - 6) * (n - 5) * (n - 4) * (n - 3) * (n - 2) * (n - 1) * (n^33 - 41025358664101065697462572830248/254239070786033029119729730175*n^32 + 3196094599361176927337311724246952/254239070786033029119729730175*n^31 - 6406072467070400380805368935285920/10169562831441321164789189207*n^30 + 1160409501417136875748705965187146596/50847814157206605823945946035*n^29 - 161970381305996685423927000792762580512/254239070786033029119729730175*n^28 + 3626387418980031371078230031993800299048/254239070786033029119729730175*n^27 - 13385611472752686345276909183338750895552/50847814157206605823945946035*n^26 + 41549775288097502968309880647760361324474/10169562831441321164789189207*n^25 - 2753009316019655888860254134942801310691824/50847814157206605823945946035*n^24 + 31524126421792019127551524627094966972464776/50847814157206605823945946035*n^23 - 62987799784499779168139187280642579637081856/10169

phi(n,45)=(4676724677977797914458050535741/18460219269364227555858512576490388127219712000000000) * (n - 8) * (n - 7) * (n - 6) * (n - 5) * (n - 4) * (n - 3) * (n - 2) * (n - 1) * (n^36 - 858716629975693254924859774090822/4676724677977797914458050535741*n^35 + 229054465303916026321275263373084847/14030174033933393743374151607223*n^34 - 13141028529958505408861159623329166336/14030174033933393743374151607223*n^33 + 182299114348787250615843172311984133452/4676724677977797914458050535741*n^32 - 5866406365574909779076894354489168833736/4676724677977797914458050535741*n^31 + 455834383602094303243423404024845884748828/14030174033933393743374151607223*n^30 - 9767076757338416699298971876253012318325424/14030174033933393743374151607223*n^29 + 58869665844998555459392619512474677324518902/4676724677977797914458050535741*n^28 - 911980898072544139176372369630504796130206532/4676724677977797914458050535741*n^27 + 12248313532841860007937502846256305419976288950/4676724677977797914458050535741*n^26 - 1

phi(n,48)=(246128192884138935357187340021030213/129311620755584090321482177576805989984598816194560000000000) * (n - 9) * (n - 8) * (n - 7) * (n - 6) * (n - 5) * (n - 4) * (n - 3) * (n - 2) * (n - 1) * (n^38 - 48647347915784871085636388041150199881/246128192884138935357187340021030213*n^37 + 4664283543594024275218707068058009717998/246128192884138935357187340021030213*n^36 - 289105780818694024064617426599077692482180/246128192884138935357187340021030213*n^35 + 13025646512655796581277219645074402445666164/246128192884138935357187340021030213*n^34 - 454781070304425957311374667685363897933679068/246128192884138935357187340021030213*n^33 + 12809792330425525456306904311176765959645420464/246128192884138935357187340021030213*n^32 - 299216368424255555098511105472922434511024172600/246128192884138935357187340021030213*n^31 + 5912994451540465379899956548790260041870107999118/246128192884138935357187340021030213*n^30 - 100360849226910245782571267519569000043469437296566/2461281928841389353571873

<h3>Section 5: NRS conjecture</h3>

In [7]:
#We implement the right hand side of theorem 5.2, thereby verifying that theorem in some examples
#b(I) is with the convention from [NRS], bshift(I) is our convention
bDict = {():1}
def b(I):
    l=len(I)
    if tuple(I) in bDict.keys():
        return bDict.get(tuple(I))
    else:
        if l==1:
            i=I[0]
            antw= (sum([binomial(n,j)*binomial(n+i-j-1,i-j) for j in [0..i]]))/(2^i)
        elif l==2:
            if I[0]==0:
                antw = b([I[1]])
            else:
                i=I[1]
                j=I[0]
                antw = b([i])*b([j])-2*sum([(-1)^(k-1)*b([i+k])*b([j-k]) for k in [1..j]])
        elif l%2==0:
            antw=sum([(-1)^(k+1)*b([I[0],I[k]])*b(I[1:k]+I[k+1:]) for k in [1..l-1]])
        else:
            if I[0]==0:
                antw=b(I[1:])
            else:
                antw=b([0]+I)
    bDict[tuple(I)]=antw
    return antw
def bshift(I):
    return b([i+1 for i in I])
def deltaNRS(m,s):
    L=[]
    for w in [0..m-s]:
        L=L+[list(el) for el in Partitions(w, length=s, max_slope=-1)]+[list(el)+[0] for el in Partitions(w, length=s-1, max_slope=-1)]
    for el in L:
        el.reverse()
    return sum([(-1)^(m-s-sum(I))*psi(I)*bshift(I)*binomial(m-1,m-s-sum(I)) for I in L])

In [8]:
#Example: we compute both sides of theorem 5.2 and indeed get the same result
m=6
s=2
print(deltaPoly(m,s))
print(deltaNRS(m,s))

11/72*n^6 - 9/8*n^5 + 185/72*n^4 - 25/24*n^3 - 49/18*n^2 + 13/6*n
11/72*n^6 - 9/8*n^5 + 185/72*n^4 - 25/24*n^3 - 49/18*n^2 + 13/6*n
